In [13]:
### pip install python-docx

In [14]:
import docx
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from string import punctuation
import string
from nltk.stem.porter import PorterStemmer
import pandas as pd
import os
import re
from textblob import TextBlob
import numpy
import pandas_datareader.data as web
import datetime
import time
from collections import Counter
from gensim import corpora

## Download packages

In [15]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jingh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jingh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jingh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jingh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Get File list in each company folder

In [16]:
def read_file(File_directory,Files,Company):
#     Copyright=[]
    date=[]
    title=[]
    company=[]
    text=[]
    for filename in Files:
    #absolute path
        filePath = File_directory+'/'+filename
        title.append(filename)
        company.append(Company)
        #read file
        doc = docx.Document(filePath)
        all_content=""
#       copyright=""
        for i in range(len(doc.paragraphs)):
            str=doc.paragraphs[i].text
            all_content=all_content+" "+str
#             if str.find("Copyright")!= -1:
#                 Copyright.append(str)
#                 #print(filename,str)
            if str.find("Load-Date")!=-1:
               date.append(str[11:])
        text.append(all_content)
    return company,date,text,title

## Create directory for each company and extract information from documents

In [17]:
IBM_directory = 'Articles/IBM'
IBMFiles = os.listdir(IBM_directory)
company_IBM,date_IBM,text_IBM,title_IBM=read_file(IBM_directory,IBMFiles,'IBM')

In [18]:
Apple_directory = 'Articles/Apple'
AppleFiles = os.listdir(Apple_directory)
company_Apple,date_Apple,text_Apple,title_Apple=read_file(Apple_directory,AppleFiles,'Apple')

In [19]:
Facebook_directory = 'Articles/facebook'
FacebookFiles = os.listdir(Facebook_directory)
company_Facebook,date_Facebook,text_Facebook,title_Facebook=read_file(Facebook_directory,FacebookFiles,'Facebook')

In [20]:
Walmart_directory = 'Articles\Walmart'
WalmartFiles = os.listdir(Walmart_directory)
company_Walmart,date_Walmart,text_Walmart,title_Walmart=read_file(Walmart_directory,WalmartFiles,'Walmart')

## Pepare data frame for company information

### Function to combine company name, pulication date, title and content to a data frame

In [21]:
def data_frame(company,date,text,title):
    data=pd.DataFrame(columns=['Company','Title','Date','content'])
    data['Company']=company
    data['Date']=date
#     data['Publisher']=Copyright
    data['content']=text
    data['Title']=title
    data = data.replace('\n','', regex=True)
    data=data.replace('\xa0', ' ',regex=True)
    data=data.replace('\xad', ' ',regex=True)
    data=data.replace('/', '')
    return data

##### IBM

In [22]:
data_IBM=data_frame(company_IBM,date_IBM,text_IBM,title_IBM)

In [23]:
data_IBM.head()

,Company,Title,Date,content
0,IBM,"Groupon, IBM Settle Patent Lawsuits In $57M De...","October 1, 2018","Benzinga October 1, 2018 Monday 5:50 PM E..."
1,IBM,"Growing pains, big dividend.docx","November 18, 2018","Spokesman Review (Spokane, WA) November 1..."
2,IBM,Growth fears drive FTSE 100 to two-year closin...,"December 18, 2018","The Guardian(London) December 18, 2018 Tu..."
3,IBM,Guyasuta Investment Advisors Inc. Reduces Posi...,"January 27, 2018","Transcript Daily January 27, 2018 Saturda..."
4,IBM,Habitat_ IBM exec's Ridgefield 'smart home' ha...,"July 20, 2018","The News-Times (Danbury, Connecticut) Jul..."


#### Apple

In [24]:
data_Apple=data_frame(company_Apple,date_Apple,text_Apple,title_Apple)

In [25]:
data_Apple.head()

,Company,Title,Date,content
0,Apple,"'Always Sunny' and Apple, Crocpocalypse averte...","August 17, 2018","RedEye, Chicago August 17, 2018 Friday C..."
1,Apple,'China's Apple' opens biggest IPO in three yea...,"May 3, 2018","Australian Financial Review May 4, 2018 F..."
2,Apple,'i had convinced myself that steve would live ...,"April 28, 2018","The Times (London) April 28, 2018 Saturda..."
3,Apple,'Infuriated' Mark Zuckerberg told Facebook sta...,"November 15, 2018","MailOnline November 15, 2018 Thursday 3:0..."
4,Apple,14 shops that would send city into the retail ...,"November 13, 2018","Nottingham Post November 13, 2018 Tuesday..."


##### Facebook

In [26]:
data_Facebook=data_frame(company_Facebook,date_Facebook,text_Facebook,title_Facebook)

In [27]:
data_Facebook.head()

,Company,Title,Date,content
0,Facebook,'A grand illusion'_ seven days that shattered ...,"June 20, 2018","The Guardian(London) March 24, 2018 Satur..."
1,Facebook,'Educational' Holocaust post deleted from Face...,"August 30, 2018","thetimes.co.uk August 31, 2018 Friday 12:..."
2,Facebook,'I made Steve Bannon's psychological warfare t...,"March 20, 2018","The Observer(London) March 17, 2018 Satur..."
3,Facebook,'I made Steve Bannon's psychological warfare t...,"June 20, 2018","The Guardian(London) March 17, 2018 Satur..."
4,Facebook,'SADNESS' FOR DEC;Follow me on Twitter_ @danwo...,"April 9, 2018","The Sun (England) April 9, 2018 Monday, E..."


##### Walmart

In [28]:
data_Walmart=data_frame(company_Walmart,date_Walmart,text_Walmart,title_Walmart)

In [29]:
data_Walmart.head()

,Company,Title,Date,content
0,Walmart,-Advent Closes Investment in 80 Percent Stake ...,"August 3, 2018","ENP Newswire August 3, 2018 Friday Copyr..."
1,Walmart,-DBRS Comments on Combination of J Sainsbury p...,"May 2, 2018","ENP Newswire May 2, 2018 Wednesday Copyr..."
2,Walmart,"1,066 Shares in Walmart Inc (WMT) Purchased by...","April 12, 2018","Zolmax.com April 12, 2018 Thursday 10:46 ..."
3,Walmart,"195,000 Shares in Walmart Inc (WMT) Purchased ...","June 30, 2018","DailyPolitical June 30, 2018 Saturday 9:3..."
4,Walmart,"195,000 Shares in Walmart Inc (WMT) Purchased ...","June 30, 2018","Week Herald June 30, 2018 Saturday 4:11 P..."


## Content Cleansing

### Stop words

In [30]:
stop_words = set(stopwords.words('english'))
stop_append = ['.', ',', '`', '"', "'", '!', ';',  '%','$','/','[',']',':','...','y/y',]
stop_word = list(stop_words.union(stop_append))

### None english words

In [31]:
pattern = re.compile(r'[0-9]+|\b[\w]{2,2}\b|[%.,_`!"&?\')({~@;:#}+-]+|\b[\w]{1,1}\b')

### Function to unify part of speech

In [32]:
def tagging(tag):   
    if tag[1].startswith('J'):   
        return nltk.stem.wordnet.wordnet.ADJ 
    elif tag[1].startswith('V'):   
        return nltk.stem.wordnet.wordnet.VERB
    elif tag[1].startswith('N'):   
        return nltk.stem.wordnet.wordnet.NOUN
    elif tag[1].startswith('R'):   
        return nltk.stem.wordnet.wordnet.ADV
    else : return nltk.stem.wordnet.wordnet.NOUN

### Function to clean document content: remove stop words and non english words.

In [33]:
def content_cleansing(df):
    df['content']=df['content'].str.lower()
    df['token']=df['content'].apply(word_tokenize)
    df['token_clean'] = df['token'].apply(lambda x: [word for word in x if word not in stop_word])
    df['token_clean']=df['token_clean'].apply(lambda x: [pattern.sub('', word) for word in x])
    df['token_clean']=df['token_clean'].apply(lambda x: [word for word in x if len(word) > 2])
    df['pos']=df['token_clean'].apply(nltk.tag.pos_tag)
    df['tagged_pos'] =df['pos'].apply(lambda x: [(t[0], tagging(t[1])) for t in x])
    df['stem']=df['tagged_pos'].apply(lambda x:[WordNetLemmatizer().lemmatize(word = t[0], pos = t[1]) for t in x])
#     df['stem_str']= df['stem'].apply([lambda x: " ".join(i for i in x)])
    df['month']=df['Date'].apply(lambda x:x[:-8])
    df['month']=df['month'].str.lstrip()
    df['month']=df['month'].str.rstrip()
    return df.head() 

##### IBM

In [34]:
 content_cleansing(data_IBM)

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month
0,IBM,"Groupon, IBM Settle Patent Lawsuits In $57M De...","October 1, 2018","benzinga october 1, 2018 monday 5:50 pm e...","[benzinga, october, 1, ,, 2018, monday, 5:50, ...","[benzinga, october, monday, est, copyright, ne...","[(benzinga, NN), (october, MD), (monday, VB), ...","[(benzinga, n), (october, n), (monday, n), (es...","[benzinga, october, monday, est, copyright, ne...",October
1,IBM,"Growing pains, big dividend.docx","November 18, 2018","spokesman review (spokane, wa) november 1...","[spokesman, review, (, spokane, ,, wa, ), nove...","[spokesman, review, spokane, november, sunday,...","[(spokesman, NN), (review, NN), (spokane, NN),...","[(spokesman, n), (review, n), (spokane, n), (n...","[spokesman, review, spokane, november, sunday,...",November
2,IBM,Growth fears drive FTSE 100 to two-year closin...,"December 18, 2018","the guardian(london) december 18, 2018 tu...","[the, guardian, (, london, ), december, 18, ,,...","[guardian, london, december, tuesday, gmt, cop...","[(guardian, JJ), (london, NN), (december, NN),...","[(guardian, a), (london, n), (december, n), (t...","[guardian, london, december, tuesday, gmt, cop...",December
3,IBM,Guyasuta Investment Advisors Inc. Reduces Posi...,"January 27, 2018","transcript daily january 27, 2018 saturda...","[transcript, daily, january, 27, ,, 2018, satu...","[transcript, daily, january, saturday, est, co...","[(transcript, JJ), (daily, JJ), (january, NN),...","[(transcript, a), (daily, a), (january, n), (s...","[transcript, daily, january, saturday, est, co...",January
4,IBM,Habitat_ IBM exec's Ridgefield 'smart home' ha...,"July 20, 2018","the news-times (danbury, connecticut) jul...","[the, news-times, (, danbury, ,, connecticut, ...","[newstimes, danbury, connecticut, july, friday...","[(newstimes, RB), (danbury, NN), (connecticut,...","[(newstimes, n), (danbury, n), (connecticut, a...","[newstimes, danbury, connecticut, july, friday...",July


##### Apple

In [35]:
content_cleansing(data_Apple)

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month
0,Apple,"'Always Sunny' and Apple, Crocpocalypse averte...","August 17, 2018","redeye, chicago august 17, 2018 friday c...","[redeye, ,, chicago, august, 17, ,, 2018, frid...","[redeye, chicago, august, friday, copyright, d...","[(redeye, NN), (chicago, NN), (august, NN), (f...","[(redeye, n), (chicago, n), (august, n), (frid...","[redeye, chicago, august, friday, copyright, d...",August
1,Apple,'China's Apple' opens biggest IPO in three yea...,"May 3, 2018","australian financial review may 4, 2018 f...","[australian, financial, review, may, 4, ,, 201...","[australian, financial, review, may, friday, p...","[(australian, JJ), (financial, JJ), (review, N...","[(australian, a), (financial, a), (review, n),...","[australian, financial, review, may, friday, p...",May
2,Apple,'i had convinced myself that steve would live ...,"April 28, 2018","the times (london) april 28, 2018 saturda...","[the, times, (, london, ), april, 28, ,, 2018,...","[times, london, april, saturday, edition, nati...","[(times, NNS), (london, VBP), (april, JJ), (sa...","[(times, n), (london, n), (april, a), (saturda...","[time, london, april, saturday, edition, natio...",April
3,Apple,'Infuriated' Mark Zuckerberg told Facebook sta...,"November 15, 2018","mailonline november 15, 2018 thursday 3:0...","[mailonline, november, 15, ,, 2018, thursday, ...","[mailonline, november, thursday, gmt, copyrigh...","[(mailonline, JJ), (november, NN), (thursday, ...","[(mailonline, a), (november, n), (thursday, a)...","[mailonline, november, thursday, gmt, copyrigh...",November
4,Apple,14 shops that would send city into the retail ...,"November 13, 2018","nottingham post november 13, 2018 tuesday...","[nottingham, post, november, 13, ,, 2018, tues...","[nottingham, post, november, tuesday, edition,...","[(nottingham, JJ), (post, NN), (november, NN),...","[(nottingham, a), (post, n), (november, n), (t...","[nottingham, post, november, tuesday, edition,...",November


##### Facebook

In [36]:
content_cleansing(data_Facebook)

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month
0,Facebook,'A grand illusion'_ seven days that shattered ...,"June 20, 2018","the guardian(london) march 24, 2018 satur...","[the, guardian, (, london, ), march, 24, ,, 20...","[guardian, london, march, saturday, gmt, copyr...","[(guardian, JJ), (london, NN), (march, NN), (s...","[(guardian, a), (london, n), (march, n), (satu...","[guardian, london, march, saturday, gmt, copyr...",June
1,Facebook,'Educational' Holocaust post deleted from Face...,"August 30, 2018","thetimes.co.uk august 31, 2018 friday 12:...","[thetimes.co.uk, august, 31, ,, 2018, friday, ...","[thetimes, august, friday, gmt, copyright, new...","[(thetimes, NNS), (august, VBP), (friday, JJ),...","[(thetimes, n), (august, n), (friday, a), (gmt...","[thetimes, august, friday, gmt, copyright, new...",August
2,Facebook,'I made Steve Bannon's psychological warfare t...,"March 20, 2018","the observer(london) march 17, 2018 satur...","[the, observer, (, london, ), march, 17, ,, 20...","[observer, london, march, saturday, gmt, copyr...","[(observer, RB), (london, VBN), (march, NN), (...","[(observer, n), (london, n), (march, n), (satu...","[observer, london, march, saturday, gmt, copyr...",March
3,Facebook,'I made Steve Bannon's psychological warfare t...,"June 20, 2018","the guardian(london) march 17, 2018 satur...","[the, guardian, (, london, ), march, 17, ,, 20...","[guardian, london, march, saturday, gmt, copyr...","[(guardian, JJ), (london, NN), (march, NN), (s...","[(guardian, a), (london, n), (march, n), (satu...","[guardian, london, march, saturday, gmt, copyr...",June
4,Facebook,'SADNESS' FOR DEC;Follow me on Twitter_ @danwo...,"April 9, 2018","the sun (england) april 9, 2018 monday, e...","[the, sun, (, england, ), april, 9, ,, 2018, m...","[sun, england, april, monday, edition, nationa...","[(sun, NN), (england, NN), (april, NN), (monda...","[(sun, n), (england, n), (april, n), (monday, ...","[sun, england, april, monday, edition, nationa...",April


#### Walmart

In [37]:
content_cleansing(data_Walmart)

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month
0,Walmart,-Advent Closes Investment in 80 Percent Stake ...,"August 3, 2018","enp newswire august 3, 2018 friday copyr...","[enp, newswire, august, 3, ,, 2018, friday, co...","[enp, newswire, august, friday, copyright, nor...","[(enp, NN), (newswire, NN), (august, NN), (fri...","[(enp, n), (newswire, n), (august, n), (friday...","[enp, newswire, august, friday, copyright, nor...",August
1,Walmart,-DBRS Comments on Combination of J Sainsbury p...,"May 2, 2018","enp newswire may 2, 2018 wednesday copyr...","[enp, newswire, may, 2, ,, 2018, wednesday, co...","[enp, newswire, may, wednesday, copyright, nor...","[(enp, JJ), (newswire, NN), (may, MD), (wednes...","[(enp, a), (newswire, n), (may, n), (wednesday...","[enp, newswire, may, wednesday, copyright, nor...",May
2,Walmart,"1,066 Shares in Walmart Inc (WMT) Purchased by...","April 12, 2018","zolmax.com april 12, 2018 thursday 10:46 ...","[zolmax.com, april, 12, ,, 2018, thursday, 10:...","[zolmaxcom, april, thursday, est, copyright, n...","[(zolmaxcom, NN), (april, NN), (thursday, NN),...","[(zolmaxcom, n), (april, n), (thursday, n), (e...","[zolmaxcom, april, thursday, est, copyright, n...",April
3,Walmart,"195,000 Shares in Walmart Inc (WMT) Purchased ...","June 30, 2018","dailypolitical june 30, 2018 saturday 9:3...","[dailypolitical, june, 30, ,, 2018, saturday, ...","[dailypolitical, june, saturday, est, copyrigh...","[(dailypolitical, JJ), (june, NN), (saturday, ...","[(dailypolitical, a), (june, n), (saturday, n)...","[dailypolitical, june, saturday, est, copyrigh...",June
4,Walmart,"195,000 Shares in Walmart Inc (WMT) Purchased ...","June 30, 2018","week herald june 30, 2018 saturday 4:11 p...","[week, herald, june, 30, ,, 2018, saturday, 4:...","[week, herald, june, saturday, est, copyright,...","[(week, NN), (herald, NN), (june, NN), (saturd...","[(week, n), (herald, n), (june, n), (saturday,...","[week, herald, june, saturday, est, copyright,...",June


## Get stock returns

In [38]:
tickers=["IBM","AAPL","FB","WMT"]

### Function to calculate monthly stock returns

In [39]:
def MonthlyReturn(tickers):

# Create an empty dataframe with tickers and date from Jan 2015
    Month = pd.date_range("2017-12-31", periods=13, freq='M')
    returns = pd.DataFrame(columns=tickers)
# download dataframe
    for i in tickers:
# Get daily data from yahoo
        df0 = web.get_data_yahoo(i, "2017-12-01", "2018-12-31")
        df1 = df0.resample('M').apply(lambda x:x[-1])
        returns[i]= df1["Adj Close"]/df1["Adj Close"].shift(1)-1
# the timer is to prevent yahoo from blocking us 
        time.sleep(10)
#returns.head()
    return returns

### Function to add month as a column for stock returns

In [40]:
def return_df (tickers):
    returns=MonthlyReturn(tickers)
    returns['month'] = returns.index
    returns=returns.reset_index(drop = True)
    returns['month']=returns['month'].apply(lambda x: x.strftime('%B'))
    returns.drop(0,axis=0,inplace=True)  
    return returns
    

In [41]:
returns=return_df(tickers)
returns.head()

,IBM,AAPL,FB,WMT,month
1,0.067006,-0.010636,0.059107,0.079494,January
2,-0.038703,0.068185,-0.045856,-0.155628,February
3,-0.015401,-0.058051,-0.103914,-0.005661,March
4,-0.055204,-0.015020,0.076413,-0.005732,April
5,-0.014358,0.135124,0.115000,-0.061044,May


### Function to merge company data, return data and sign value

In [42]:
def company_return(comp_df,returns,company_stock):
    company_return=pd.merge(comp_df,returns[[company_stock,'month']],on='month')
    company_return["sign function"]=company_return[company_stock].apply([lambda x: 0 if x<0 else 1])
    return company_return

##### Apple

In [43]:
apple_return=company_return(data_Apple,returns,'AAPL')
apple_return.head()

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month,AAPL,sign function
0,Apple,"'Always Sunny' and Apple, Crocpocalypse averte...","August 17, 2018","redeye, chicago august 17, 2018 friday c...","[redeye, ,, chicago, august, 17, ,, 2018, frid...","[redeye, chicago, august, friday, copyright, d...","[(redeye, NN), (chicago, NN), (august, NN), (f...","[(redeye, n), (chicago, n), (august, n), (frid...","[redeye, chicago, august, friday, copyright, d...",August,0.200422,1
1,Apple,Apple becomes first US trillion dollar company...,"August 3, 2018","the times (london) august 3, 2018 friday,...","[the, times, (, london, ), august, 3, ,, 2018,...","[times, london, august, friday, edition, natio...","[(times, NNS), (london, JJ), (august, JJ), (fr...","[(times, n), (london, a), (august, a), (friday...","[time, london, august, friday, edition, nation...",August,0.200422,1
2,Apple,Apple becomes first US trillion dollar company...,"August 3, 2018","the times (london) august 3, 2018 friday,...","[the, times, (, london, ), august, 3, ,, 2018,...","[times, london, august, friday, edition, irela...","[(times, NNS), (london, JJ), (august, JJ), (fr...","[(times, n), (london, a), (august, a), (friday...","[time, london, august, friday, edition, irelan...",August,0.200422,1
3,Apple,Apple becomes first US trillion dollar company...,"August 3, 2018","the times (london) august 3, 2018 friday,...","[the, times, (, london, ), august, 3, ,, 2018,...","[times, london, august, friday, edition, scotl...","[(times, NNS), (london, JJ), (august, JJ), (fr...","[(times, n), (london, a), (august, a), (friday...","[time, london, august, friday, edition, scotla...",August,0.200422,1
4,Apple,Apple shares open 4.4% higher as its value hea...,"August 1, 2018","the guardian(london) august 1, 2018 wedne...","[the, guardian, (, london, ), august, 1, ,, 20...","[guardian, london, august, wednesday, gmt, cop...","[(guardian, JJ), (london, NN), (august, NN), (...","[(guardian, a), (london, n), (august, n), (wed...","[guardian, london, august, wednesday, gmt, cop...",August,0.200422,1


##### IBM

In [44]:
IBM_return=company_return(data_IBM,returns,'IBM')
IBM_return.head()

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month,IBM,sign function
0,IBM,"Groupon, IBM Settle Patent Lawsuits In $57M De...","October 1, 2018","benzinga october 1, 2018 monday 5:50 pm e...","[benzinga, october, 1, ,, 2018, monday, 5:50, ...","[benzinga, october, monday, est, copyright, ne...","[(benzinga, NN), (october, MD), (monday, VB), ...","[(benzinga, n), (october, n), (monday, n), (es...","[benzinga, october, monday, est, copyright, ne...",October,-0.236625,0
1,IBM,Hamilton-born Red Hat founder Bob Young reflec...,"October 29, 2018",breaking news from globeandmail.com octob...,"[breaking, news, from, globeandmail.com, octob...","[breaking, news, globeandmailcom, october, mon...","[(breaking, VBG), (news, NN), (globeandmailcom...","[(breaking, n), (news, n), (globeandmailcom, n...","[breaking, news, globeandmailcom, october, mon...",October,-0.236625,0
2,IBM,"Head of IBM Watson Health Steps Down, STAT Rep...","October 23, 2018","head of ibm watson health steps down, stat...","[head, of, ibm, watson, health, steps, down, ,...","[head, ibm, watson, health, steps, stat, repor...","[(head, NN), (ibm, NN), (watson, NN), (health,...","[(head, n), (ibm, n), (watson, n), (health, n)...","[head, ibm, watson, health, step, stat, report...",October,-0.236625,0
3,IBM,IBM Acquires Redhat_ Now What_.docx,"October 30, 2018","gurufocus.com october 29, 2018 monday 8:0...","[gurufocus.com, october, 29, ,, 2018, monday, ...","[gurufocuscom, october, monday, est, copyright...","[(gurufocuscom, NN), (october, MD), (monday, V...","[(gurufocuscom, n), (october, n), (monday, n),...","[gurufocuscom, october, monday, est, copyright...",October,-0.236625,0
4,IBM,"IBM Bond Yields, Default Risk Spike After Red ...","October 29, 2018","phil's stock world october 29, 2018 monda...","[phil, 's, stock, world, october, 29, ,, 2018,...","[phil, stock, world, october, monday, est, cop...","[(phil, NN), (stock, NN), (world, NN), (octobe...","[(phil, n), (stock, n), (world, n), (october, ...","[phil, stock, world, october, monday, est, cop...",October,-0.236625,0


##### Facebook

In [45]:
Facebook_return=company_return(data_Facebook,returns,'FB')
Facebook_return.head()

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month,FB,sign function
0,Facebook,'A grand illusion'_ seven days that shattered ...,"June 20, 2018","the guardian(london) march 24, 2018 satur...","[the, guardian, (, london, ), march, 24, ,, 20...","[guardian, london, march, saturday, gmt, copyr...","[(guardian, JJ), (london, NN), (march, NN), (s...","[(guardian, a), (london, n), (march, n), (satu...","[guardian, london, march, saturday, gmt, copyr...",June,0.013244,1
1,Facebook,'I made Steve Bannon's psychological warfare t...,"June 20, 2018","the guardian(london) march 17, 2018 satur...","[the, guardian, (, london, ), march, 17, ,, 20...","[guardian, london, march, saturday, gmt, copyr...","[(guardian, JJ), (london, NN), (march, NN), (s...","[(guardian, a), (london, n), (march, n), (satu...","[guardian, london, march, saturday, gmt, copyr...",June,0.013244,1
2,Facebook,A Risky Bet on Instagram Is Paying Off for Fac...,"June 28, 2018","the new york times june 28, 2018 thursday...","[the, new, york, times, june, 28, ,, 2018, thu...","[new, york, times, june, thursday, late, editi...","[(new, JJ), (york, NN), (times, NNS), (june, J...","[(new, a), (york, n), (times, n), (june, a), (...","[new, york, time, june, thursday, late, editio...",June,0.013244,1
3,Facebook,AT SENATE COMMERCE COMMITTEE HEARING ON CAMBRI...,"June 21, 2018","states news service june 20, 2018 wednesd...","[states, news, service, june, 20, ,, 2018, wed...","[states, news, service, june, wednesday, copyr...","[(states, NNS), (news, NN), (service, NN), (ju...","[(states, n), (news, n), (service, n), (june, ...","[state, news, service, june, wednesday, copyri...",June,0.013244,1
4,Facebook,Bug made private posts of 14mn Facebook users ...,"June 8, 2018","ians-english june 8, 2018 friday copyrig...","[ians-english, june, 8, ,, 2018, friday, copyr...","[iansenglish, june, friday, copyright, ffc, in...","[(iansenglish, JJ), (june, NN), (friday, NN), ...","[(iansenglish, a), (june, n), (friday, n), (co...","[iansenglish, june, friday, copyright, ffc, in...",June,0.013244,1


#### Walmart

In [46]:
Walmart_return=company_return(data_Walmart,returns,'WMT')
Walmart_return.head()

,Company,Title,Date,content,token,token_clean,pos,tagged_pos,stem,month,WMT,sign function
0,Walmart,-Advent Closes Investment in 80 Percent Stake ...,"August 3, 2018","enp newswire august 3, 2018 friday copyr...","[enp, newswire, august, 3, ,, 2018, friday, co...","[enp, newswire, august, friday, copyright, nor...","[(enp, NN), (newswire, NN), (august, NN), (fri...","[(enp, n), (newswire, n), (august, n), (friday...","[enp, newswire, august, friday, copyright, nor...",August,0.080542,1
1,Walmart,Advent Closes Investment in 80 Percent Stake i...,"August 2, 2018","business wire august 1, 2018 wednesday 8:...","[business, wire, august, 1, ,, 2018, wednesday...","[business, wire, august, wednesday, gmt, copyr...","[(business, NN), (wire, NN), (august, NN), (we...","[(business, n), (wire, n), (august, n), (wedne...","[business, wire, august, wednesday, gmt, copyr...",August,0.080542,1
2,Walmart,Advisory Services Network LLC Has $4.15 Millio...,"August 8, 2018","dailypolitical august 7, 2018 tuesday 10:...","[dailypolitical, august, 7, ,, 2018, tuesday, ...","[dailypolitical, august, tuesday, est, copyrig...","[(dailypolitical, JJ), (august, NN), (tuesday,...","[(dailypolitical, a), (august, n), (tuesday, n...","[dailypolitical, august, tuesday, est, copyrig...",August,0.080542,1
3,Walmart,"American Century Companies Inc. Sells 1,379,34...","August 31, 2018","week herald august 31, 2018 friday 9:09 a...","[week, herald, august, 31, ,, 2018, friday, 9:...","[week, herald, august, friday, est, copyright,...","[(week, NN), (herald, NN), (august, RB), (frid...","[(week, n), (herald, n), (august, n), (friday,...","[week, herald, august, friday, est, copyright,...",August,0.080542,1
4,Walmart,Arizona State Retirement System Grows Holdings...,"August 28, 2018","theolympiareport august 28, 2018 tuesday ...","[theolympiareport, august, 28, ,, 2018, tuesda...","[theolympiareport, august, tuesday, est, copyr...","[(theolympiareport, NN), (august, NN), (tuesda...","[(theolympiareport, n), (august, n), (tuesday,...","[theolympiareport, august, tuesday, est, copyr...",August,0.080542,1


## Screening for sentiment-charged words

### get all document frequency

In [47]:
stem_all=pd.concat([apple_return[['stem','sign function']],IBM_return[['stem','sign function']],Facebook_return[['stem','sign function']],Walmart_return[['stem','sign function']]])
stem_all.head()

,stem,sign function
0,"[redeye, chicago, august, friday, copyright, d...",1
1,"[time, london, august, friday, edition, nation...",1
2,"[time, london, august, friday, edition, irelan...",1
3,"[time, london, august, friday, edition, scotla...",1
4,"[guardian, london, august, wednesday, gmt, cop...",1


### Calcuate document frequencies of each word

In [48]:
dictionary = corpora.Dictionary(stem_all['stem'])
word_dic={}
for key,value in dictionary.dfs.items():
    word=dictionary[key]
    word_dic[word]=value

### Assume κ=100 and get document frequncies for all words

In [49]:
for word in word_dic.copy():
    if word_dic[word]<100:
        word_dic.pop(word)
word_dic
    

{'ability': 161,
 'able': 254,
 'access': 240,
 'accessed': 129,
 'according': 770,
 'account': 284,
 'accused': 126,
 'acquired': 172,
 'acquiring': 198,
 'acquisition': 114,
 'across': 260,
 'act': 175,
 'action': 281,
 'activity': 165,
 'actually': 162,
 'ad': 138,
 'add': 114,
 'added': 397,
 'adding': 120,
 'addition': 117,
 'additional': 528,
 'address': 301,
 'administration': 113,
 'advertising': 169,
 'advisor': 175,
 'affect': 100,
 'affected': 118,
 'age': 112,
 'agency': 164,
 'ago': 239,
 'agreed': 112,
 'agreement': 144,
 'ahead': 182,
 'alice': 129,
 'allow': 165,
 'allowed': 160,
 'allowing': 101,
 'almost': 183,
 'along': 151,
 'already': 290,
 'also': 1292,
 'although': 134,
 'always': 154,
 'amazon': 215,
 'america': 302,
 'american': 424,
 'among': 285,
 'amount': 204,
 'analysis': 111,
 'analyst': 650,
 'analytica': 223,
 'analytics': 147,
 'announced': 427,
 'announcement': 103,
 'annual': 128,
 'annualized': 103,
 'another': 352,
 'answer': 117,
 'anticipate': 10

### Get words in documents which have positive returns

In [50]:
stem_all_pos=stem_all[stem_all['sign function']==1]

### Document frequency for sgn funtion=1

In [51]:
doclist_pos=stem_all_pos['stem']
dictionary_pos = corpora.Dictionary(doclist_pos)
word_dic_pos={}
for key,value in dictionary_pos.dfs.items():
    word=dictionary_pos[key]
    word_dic_pos[word]=value

### Assume κ=100 and get document frequncies for all words related with positive returns

In [52]:
for i in word_dic_pos.copy():
    if(word_dic_pos[i]<100):
        word_dic_pos.pop(i)
word_dic_pos

{'according': 301,
 'account': 124,
 'across': 108,
 'action': 104,
 'added': 134,
 'additional': 221,
 'address': 126,
 'already': 115,
 'also': 503,
 'america': 138,
 'american': 156,
 'among': 100,
 'analyst': 253,
 'announced': 192,
 'another': 116,
 'apple': 137,
 'application': 127,
 'approximately': 167,
 'april': 158,
 'around': 188,
 'article': 118,
 'assigned': 101,
 'august': 181,
 'available': 145,
 'average': 235,
 'back': 147,
 'bank': 200,
 'based': 134,
 'basis': 116,
 'beta': 168,
 'big': 194,
 'billion': 361,
 'body': 680,
 'bonoboscom': 110,
 'bought': 115,
 'business': 434,
 'buy': 246,
 'buying': 108,
 'byline': 335,
 'call': 116,
 'called': 123,
 'canada': 121,
 'capital': 162,
 'carry': 131,
 'case': 150,
 'cash': 167,
 'ceo': 138,
 'change': 242,
 'chief': 163,
 'clear': 100,
 'club': 157,
 'clubs': 111,
 'come': 152,
 'commerce': 163,
 'commission': 212,
 'company': 582,
 'compared': 215,
 'consensus': 184,
 'consumer': 104,
 'content': 142,
 'convenience': 120

### Calcuate fi

In [53]:
fi={}
for word in word_dic_pos:
    pos=word_dic_pos[word]
    full=word_dic[word]
    fi[word]=pos/full
fi

{'according': 0.39090909090909093,
 'account': 0.43661971830985913,
 'across': 0.4153846153846154,
 'action': 0.3701067615658363,
 'added': 0.33753148614609574,
 'additional': 0.4185606060606061,
 'address': 0.4186046511627907,
 'already': 0.39655172413793105,
 'also': 0.3893188854489164,
 'america': 0.45695364238410596,
 'american': 0.36792452830188677,
 'among': 0.3508771929824561,
 'analyst': 0.3892307692307692,
 'announced': 0.4496487119437939,
 'another': 0.32954545454545453,
 'apple': 0.2778904665314402,
 'application': 0.4471830985915493,
 'approximately': 0.5,
 'april': 0.5015873015873016,
 'around': 0.3533834586466165,
 'article': 0.4591439688715953,
 'assigned': 0.42436974789915966,
 'august': 0.530791788856305,
 'available': 0.4005524861878453,
 'average': 0.4484732824427481,
 'back': 0.3838120104438642,
 'bank': 0.4694835680751174,
 'based': 0.38285714285714284,
 'basis': 0.39862542955326463,
 'beta': 0.4253164556962025,
 'big': 0.3527272727272727,
 'billion': 0.41589861751

### I assume fi>0.45 is positive and fi<0.35 is negative and remove word with fi=1

In [54]:
for i in fi.copy():
    if (fi[i]==1.0):
        fi.pop(i)

In [55]:
pos=[]
neg=[]
for i in fi:
    if(fi[i]>0.45):
        pos.append(i)
    if(fi[i]<0.35):
        neg.append(i)

#### postive words

In [56]:
pos

['august',
 'content',
 'bank',
 'may',
 'visit',
 'america',
 'valued',
 'mobile',
 'shareholder',
 'page',
 'july',
 'following',
 'private',
 'expectation',
 'major',
 'solution',
 'purchasing',
 'june',
 'clear',
 'april',
 'retail',
 'total',
 'commerce',
 'club',
 'previous',
 'trust',
 'disclosed',
 'cash',
 'grew',
 'retailer',
 'article',
 'worldwide',
 'net',
 'website',
 'sec',
 'carry',
 'transaction',
 'various',
 'walmart',
 'sam',
 'warehouse',
 'discount',
 'approximately',
 'disclosure',
 'voiceactivated',
 'convenience',
 'engages',
 'supermarket',
 'walton',
 'supercenters',
 'hypermarket',
 'clubs',
 'drugstore',
 'membershiponly',
 'walmartcom',
 'jetcom',
 'hayneedlecom',
 'shoescom',
 'moosejawcom',
 'modclothcom',
 'bonoboscom',
 'samsclubcom']

##### negative words

In [57]:
neg

['apple',
 'record',
 'sell',
 'executive',
 'early',
 'software',
 'consumer',
 'said',
 'move',
 'next',
 'another',
 'february',
 'september',
 'told',
 'york',
 'tech',
 'added',
 'partner',
 'daily',
 'november',
 'october']